In [1]:
# Load Libraries
import warnings
from datetime import datetime

import tensorflow as tf
from keras import backend as K
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding, GRU, Input
from keras.layers.core import Dense, Dropout
from keras.models import Sequential, Model
from keras.optimizers import Adam
from model_evaluator import Evaluator
from model_preprocessor import Preprocessor

warnings.filterwarnings("ignore")
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
K.tensorflow_backend.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
 def gru(max_len=73, emb_dim=32, max_vocab_len=40, W_reg=regularizers.l2(1e-4)):
    """GRU model with the Keras Sequential model"""
    input = Input(shape=(max_len,), dtype='int32', name='gru_input')
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len, W_regularizer=W_reg)(input)
    emb = Dropout(0.5)(emb)
    gru = GRU(128)(emb)
    gru = Dropout(0.5)(gru)
    output = Dense(21, activation='softmax', name='gru_output')(gru)
    model = Model(input=[input], output=[output])
    
    return model

In [3]:
# Load data using model preprocessor
x_train, x_test, y_train, y_test = Preprocessor.load_data()

In [4]:
# Define Deep Learning Model
model_name = "GRU"
model = gru()

''' Training phrase '''
epochs = 5
batch_size = 64
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.CategoricalAccuracy(),
                       Evaluator.precision, Evaluator.recall, Evaluator.fmeasure])

model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.11)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 1149965 samples, validate on 142131 samples
Epoch 1/5
1149965/1149965 [==============================] - 1016s 884us/step - loss: 0.8377 - accuracy: 0.7486 - categorical_accuracy: 0.6513 - precision: 0.8525 - recall: 0.6708 - fmeasure: 0.7469 - val_loss: 0.4669 - val_accuracy: 0.8613 - val_categorical_accuracy: 0.7551 - val_precision: 0.9297 - val_recall: 0.8105 - val_fmeasure: 0.8655
Epoch 2/5
1149965/1149965 [==============================] - 1015s 883us/step - loss: 0.4273 - accuracy: 0.8749 - categorical_accuracy: 0.7907 - precision: 0.9252 - recall: 0.8337 - fmeasure: 0.8766 - val_loss: 0.2838 - val_accuracy: 0.9161 - val_categorical_accuracy: 0.8175 - val_precision: 0.9432 - val_recall: 0.8932 - val_fmeasure: 0.9173
Epoch 3/5
1149965/1149965 [==============================] - 1015s 882us/step - loss: 0.3009 - accuracy: 0.9122 - categorical_accuracy: 0.8

In [5]:
model_json = model.to_json()
with open("./saved_model/gru.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("./saved_model/gru.h5")